In [7]:
!pip install transformers

In [8]:
!pip install torch

In [9]:
!nvidia-smi

Fri Nov 24 01:08:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    30W /  70W |   2245MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
!pip install fugashi
!pip install mecab-python3
!pip install unidic-lite

In [22]:
!pip install ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=60b7989a43083eca9fd91b2b6657428ba32c0ed2b8842dbd6b9064740766ed56
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic


In [30]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("nlp-waseda/bigbird-base-japanese")
model = AutoModel.from_pretrained("nlp-waseda/bigbird-base-japanese")

tokenizer_config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/810k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/228M [00:00<?, ?B/s]

In [37]:
import torch
import time

# Set the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print (device)

# Define a function to classify a text and get CLS's embedding
def get_cls_embedding_and_tokenized_output(input_corpus):
    # Tokenize input and get token outputs
    encoded_input_corpus  = tokenizer(input_corpus, return_tensors="pt", truncation=True)
    encoded_input_corpus = {key: val.to(device) for key, val in encoded_input_corpus.items()}

    # Run the model and get CLS's embedding
    start_time = time.time()
    with torch.no_grad():
        output  = model(**encoded_input_corpus )
        cls_embedding = output.last_hidden_state[:, 0]
        # Convert the tokenized output to a list of tokens
        tokenized_output = tokenizer.convert_ids_to_tokens(encoded_input_corpus["input_ids"].squeeze().cpu().numpy())
    elapsed_time = time.time() - start_time
    return cls_embedding, tokenized_output, elapsed_time, encoded_input_corpus

#input_corpus = "これは日本語のテキストのコープスです。"
input_corpus = """
[オペレーター] : こんにちは！お電話ありがとうございます。私の名前はエミリーで、今日はお手伝いできて嬉しいです。どのようにお手伝いできますか？
[ユーザー] : こんにちは、エミリー。貴社のサービスに興味がありますが、それが私にとって適しているかどうか分かりません。
[オペレーター] : 完全に理解していますし、興味を持っていただきありがとうございます！様々なニーズに対応した幅広いサービスを提供しています。サービスに関してもう少し教えていただけますか？
[ユーザー] : そうですね、信頼性があり、手頃な価格のものが必要です。過去にサービスで悪い経験をしました。
[オペレーター] : 了解しました。当社はすべてのサービスで信頼性と手頃な価格を重視しています。パッケージについての詳細を共有させていただきます。現在、新規顧客向けに割引料金が適用される特別なプロモーションを実施しています。これにより、高品質なサービスを手頃な価格でご利用いただけます。
[ユーザー] : 興味深いですね。サービスの具体的な機能について教えていただけますか？
[オペレーター] : もちろんです！当社のサービスには [機能1]、[機能2]、および [機能3] が含まれており、お客様の特定のニーズに応じて設計されています。これらの機能は非常に有益だと感じている現在のお客様からも優れたフィードバックをいただいています。
[ユーザー] : うーん、有望ですね。でもまだ自分に合っているかどうか分かりません。
[オペレーター] : お気持ち理解しています。迷われているのは分かります。お客様の決断をサポートするために、30日間の返金保証もご用意しています。これにより、1か月間はリスクを取らずにサービスを試していただけます。期待通りでない場合は、全額返金いたします。お気軽にお試しください。
[ユーザー]: それは寛大な提案ですね。保証について感謝します。
"""
cls_embedding, tokenized_output, elapsed_time, encoded_input_corpus = get_cls_embedding_and_tokenized_output(input_corpus)

print("CLS embedding's first 20 elements:", cls_embedding.squeeze()[:20].tolist())
print("CLS embedding size:", cls_embedding.size())
print("Inference time:", elapsed_time)
print("Input corpus: ", input_corpus)
print("Tokenized output:", tokenized_output)
print("Tokenized output length:", len(tokenized_output))
# Print encoded_input_corpus properties
print("Input IDs:", encoded_input_corpus["input_ids"])
print("Attention Mask:", encoded_input_corpus["attention_mask"])
# Compare with traditional BERT, there is no property "token_type_ids" so remove this code below
#print("Token Type IDs:", encoded_input_corpus["token_type_ids"])

cuda
CLS embedding's first 20 elements: [0.1274779587984085, -0.038660552352666855, -0.022486016154289246, -0.13733258843421936, -0.16573674976825714, -0.09585399925708771, 0.25447869300842285, 0.3493923246860504, -0.054612573236227036, -0.4857536256313324, -0.21477608382701874, -0.2535403072834015, -0.1198803037405014, 0.13311977684497833, -0.0021423606667667627, 0.16039268672466278, -0.14710132777690887, -0.1327587366104126, -0.031121717765927315, -0.03668970614671707]
CLS embedding size: torch.Size([1, 768])
Inference time: 0.0675191879272461
Input corpus:  
[オペレーター] : こんにちは！お電話ありがとうございます。私の名前はエミリーで、今日はお手伝いできて嬉しいです。どのようにお手伝いできますか？
[ユーザー] : こんにちは、エミリー。貴社のサービスに興味がありますが、それが私にとって適しているかどうか分かりません。
[オペレーター] : 完全に理解していますし、興味を持っていただきありがとうございます！様々なニーズに対応した幅広いサービスを提供しています。サービスに関してもう少し教えていただけますか？
[ユーザー] : そうですね、信頼性があり、手頃な価格のものが必要です。過去にサービスで悪い経験をしました。
[オペレーター] : 了解しました。当社はすべてのサービスで信頼性と手頃な価格を重視しています。パッケージについての詳細を共有させていただきます。現在、新規顧客向けに割引料金が適用される特別なプロモーションを実施しています。これにより、高品質なサービスを手頃な価格でご利用いただけます。
[ユ